In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip '/content/drive/My Drive/Leukocyte/blood-cells.zip' -d '/content/drive/My Drive/Leukocyte/blood-cells/'

Archive:  /content/drive/My Drive/Leukocyte/blood-cells.zip
replace /content/drive/My Drive/Leukocyte/blood-cells/dataset-master/dataset-master/Annotations/BloodImage_00000.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
import numpy as np
import keras
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Bidirectional, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization, Input, concatenate, Reshape, LSTM, CuDNNLSTM
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.utils.np_utils import to_categorical
from keras import backend as K
import os
import cv2
import scipy
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import skimage
import sklearn
from tqdm import tqdm

In [0]:
print(os.listdir("/content/drive/My Drive/Leukocyte/blood-cells"))

In [0]:
class1 = {1:'NEUTROPHIL',2:'EOSINOPHIL',3:'MONOCYTE',4:'LYMPHOCYTE'}
class2 = {0:'Mononuclear',1:'Polynuclear'}

In [0]:
def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    z = []
    for wbc_type in os.listdir(folder):
        if not wbc_type.startswith('.'):
            if wbc_type in ['NEUTROPHIL']:
                label = 1
                label2 = 1
            elif wbc_type in ['EOSINOPHIL']:
                label = 2
                label2 = 1
            elif wbc_type in ['MONOCYTE']:
                label = 3  
                label2 = 0
            elif wbc_type in ['LYMPHOCYTE']:
                label = 4 
                label2 = 0
            else:
                label = 5
                label2 = 0
            for image_filename in tqdm(os.listdir(folder + wbc_type)):
                img_file = cv2.imread(folder + wbc_type + '/' + image_filename)
                if img_file is not None:
                    img_file = cv2.resize(img_file, dsize=(80,60), interpolation=cv2.INTER_CUBIC)
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    z.append(label2)
    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(z)
    return X,y,z

In [0]:
X_train, y_train, z_train = get_data('/content/drive/My Drive/Leukocyte/blood-cells/dataset2-master/dataset2-master/images/TRAIN/')

In [0]:
X_test, y_test, z_test = get_data('/content/drive/My Drive/Leukocyte/blood-cells/dataset2-master/dataset2-master/images/TEST/')

In [0]:
# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
from keras.utils.np_utils import to_categorical
y_trainHot = to_categorical(y_train, num_classes = 5)
y_testHot = to_categorical(y_test, num_classes = 5)
z_trainHot = to_categorical(z_train, num_classes = 2)
z_testHot = to_categorical(z_test, num_classes = 2)
dict_characters = {1:'NEUTROPHIL',2:'EOSINOPHIL',3:'MONOCYTE',4:'LYMPHOCYTE'}
dict_characters2 = {0:'Mononuclear',1:'Polynuclear'}
print(dict_characters)
print(dict_characters2) 

In [0]:
print("Train X Shape --> ",X_train.shape)
print("Train y Shape --> ",y_trainHot.shape)
print("Train z Shape --> ",z_trainHot.shape)

In [0]:
X_train = X_train/255.0
X_test = X_test/255.0

In [0]:
import seaborn as sns
sns.countplot(y_train,palette='Set2')

# CNN Model

In [0]:
model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(60,80,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.20))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.20))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.40))

model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
          
model.add(Dense(5,activation='softmax'))

model.summary()

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
hist=model.fit(X_train,y_trainHot,epochs=50,batch_size=128,validation_split=0.2)

# RNN Model

In [0]:
def grayscale(input):
    """Averaging out each pixel in its 3 RGB layers that results in a grayscale image"""
    return K.mean(input, axis=3)
def grayscale_shape(input):
    return input[:-1]

In [0]:
def CNNnRNN(v,w,x,y,z, epochs, classes):
    
    batch_size = 128
    num_classes = len(w[0])
    #img_rows, img_cols = v.shape[1], v.shape[2]
    img_rows,img_cols=60,80
    input_shape = (img_rows, img_cols, 3)
    ipt_tensor = Input(shape=input_shape)
    
    #CNN
    modelcnn = Sequential()
    
    modelcnn.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape,strides=z))
    modelcnn.add(Conv2D(64, (3, 3), activation='relu'))
    modelcnn.add(MaxPool2D(pool_size=(2, 2)))
    modelcnn.add(Dropout(0.25))
    modelcnn.add(Flatten())
    
    cnn_output = modelcnn(ipt_tensor)
    
    # RNN    
    modelrnn = Lambda(grayscale, grayscale_shape)(ipt_tensor)
    
    #modelrnn = Bidirectional(LSTM(64, return_sequences=True, dropout=0.25, recurrent_dropout=0.25))(modelrnn)
    #rnn_output = Bidirectional(LSTM(64, dropout=0.25, recurrent_dropout=0.25))(modelrnn)
    
    modelrnn = Bidirectional(CuDNNLSTM(64, return_sequences=True))(modelrnn)
    modelrnn = Dropout(0.25)(modelrnn)
    
    rnn_output = Bidirectional(CuDNNLSTM(64))(modelrnn)
    rnn_output = Dropout(0.25)(rnn_output)
    
    # Merging both cnn bottleneck and rnn's output wise element wise multiplication
    
    merge_layer = concatenate([cnn_output, rnn_output])
    
    last = Dense(128, activation='relu')(merge_layer)
    last = Dropout(0.5)(last)
    
    output_tensor = Dense(num_classes, activation='softmax')(last)
    
    model = Model(input=ipt_tensor, output=output_tensor)
    
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])    
    
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zoom_range=0.1,
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False
        )  # randomly flip images
    
    history = model.fit_generator(datagen.flow(v,w, batch_size=32,shuffle=True), steps_per_epoch=len(v) / 32, epochs=epochs, validation_data = [x,y])
    
    score = model.evaluate(x,y, verbose=0)
    print('\nKeras CNN+RNN accuracy:', score[1],'\n')
    
    y_pred = model.predict(x)
    
    map_characters = classes
    
    print('\n', sklearn.metrics.classification_report(np.where(y > 0)[1], np.argmax(y_pred, axis=1), target_names=list(map_characters.values())), sep='')    
    Y_pred_classes = np.argmax(y_pred,axis=1) 
    Y_true = np.argmax(y,axis=1) 

In [0]:
leukocyte = CNNnRNN(X_train,y_trainHot,X_test,y_testHot,1,100, class1)

In [0]:
model.save_weights('/content/drive/My Drive/Leukocyte/leu.h5')